In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
HousePriceTrain=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
HousePriceTest=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

y=HousePriceTrain['SalePrice']
HousePriceTrain=HousePriceTrain.drop(['SalePrice'],axis=1)
Ids=HousePriceTest['Id']

FullData=pd.concat([HousePriceTrain,HousePriceTest])
#HousePriceTrain.select_dtypes(exclude=['number','MiscFeature','Fence','PoolQC']).isnull().sum()

FullData=FullData.drop(['Alley','PoolQC','Fence','MiscFeature','Id', 'BsmtCond', 'BsmtExposure', 'CentralAir', 'Condition1','ExterCond',  'Exterior1st','Functional','GarageQual','HouseStyle',  'LandSlope', 'LotConfig','LotShape', 'MSZoning','SaleType', 'Street', 'Utilities'],axis=1)
#print(FullData.Exterior2nd.unique())
FullData.info()
FullData.fillna(FullData.mean(), inplace=True)
#FullData.fillna(FullData.mode, inplace=True)
#FullData.select_dtypes(exclude=['number']).isnull().sum()
#CategoricalCols=FullData.select_dtypes(exclude=['number']).columns
#CategoricalFrame=FullData.select_dtypes(exclude=['number']).columns
#print(CategoricalFrame)

CategoricalCols=['LandContour', 
         'Neighborhood', 'Condition2',
       'BldgType',  'RoofStyle', 'RoofMatl', 
       'Exterior2nd', 'MasVnrType', 'ExterQual','Foundation',
       'BsmtQual', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC',  'Electrical', 'KitchenQual',
        'FireplaceQu', 'GarageType', 'GarageFinish', 
       'GarageCond', 'PavedDrive', 'SaleCondition']
IndicatorVaribles=['HLS',
'BrDale',
'Edwards', 
'Timber',
'PosA',
'PosN',
'1Fam',
'TwnhsE',
'Shed',
'WdShngl',
'Stucco',
'Stone',
'Ex',
'PConc',
'GLQ',
'GasA',
'SBrkr',
'BuiltIn',
'Fin',
'Partial']
def encode(y):
 return 1 if y in IndicatorVaribles  else 0
#train['enc_condition'] = train.SaleCondition.apply(encode)
#test['enc_condition'] = test.SaleCondition.apply(encode)
for column in CategoricalCols:
    FullData[column].fillna(FullData[column].mode()[0], inplace=True)
    
    
for column in CategoricalCols:
    #FullData = FullData.apply(lambda x: encode(x) if x.name == 'SaleCondition' else x)
    FullData[column] = FullData[column].apply(encode)
    
#print(FullData['SaleCondition'])


#FullData = pd.get_dummies(FullData, columns=CategoricalCols, drop_first=True)

#print(FullData.head())
#FullData.head()
df1 = FullData.loc[:1459]
df2 = FullData.iloc[1460:]


df1.to_csv('df1.csv')
df2.to_csv('df2.csv')
#CategoricalFrame.describe()

In [ ]:
#Training the model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor  
from xgboost import XGBClassifier
#regressor = XGBClassifier()
X_train,X_test,y_train,y_test=train_test_split(df1,y,test_size=0.3,random_state=0)
#regressor = DecisionTreeRegressor(random_state = 0)
regressor = LinearRegression()  

regressor.fit(X_train,y_train)
predictions=regressor.predict(X_test)
#results_df=pd.DataFrame()
print(mean_squared_error(y_test,predictions))

results_df['Id']=Ids
results_df['SalePrice']=predictions

results_df.info()
results_df.head()
results_df.set_index(['Id'], inplace = True) 
results_df.to_csv("submission.csv")
#mean_squared_error(y_test,predictions)
#print(mean_squared_error(y_test,predictions))

In [ ]:
import matplotlib.pyplot as plt
for x in CategoricalCols:
    condition_pivot = HousePriceTrain.pivot_table(index=x, values='SalePrice', aggfunc=np.median)
    condition_pivot.plot(kind='bar', color='blue')
    plt.xlabel(x)
    plt.ylabel('Median Sale Price')
    plt.xticks(rotation=0)
    plt.show()